# CS 447 Homework 1 $-$ Language Models & Morphological Transduction
In this homework we will study some traditional appraoches to a few natural language tasks. First, you will build some n-gram language models on a corpus of Wikipedia articles, and then you will design a finite-state transducer for verb conjugation in Spanish.

This notebook is designed to be run in Google Colab. Navigate to <TT>colab.research.google.com</TT> and upload this notebook. Then follow the instructions in the notebook to do the assignent.

To run the notebook, you will need to connect to a Runtime. For this homework, all you need is a CPU. You can change the runtime by going to <TT>Runtime > Change runtime type</TT> and selecting <TT>None</TT> in the <TT>Hardware Accelerator</TT> field. We encourage you to disconnect from the runtime when you are not using it, as Google Colab can limit your resources if you overuse them.

You can read more about Google Colab at https://research.google.com/colaboratory/faq.html.

We have imported all the libraries you need to do this homework. <b>You should not import any extra libraries.</b> If you do, the autograder will fail to run your code.

#Part 1: Language Models [60 points]

Here, you will train some <b>n-gram language models</b> on WikiText-2, a corpus of high-quality Wikipedia articles. The dataset was originally introduced in the following paper: https://arxiv.org/pdf/1609.07843v1.pdf. A raw version of the data can easily be viewed here: https://github.com/pytorch/examples/tree/master/word_language_model/data/wikitext-2.

Unfortunately, you have to install the <TT>torchdata</TT> package on the Colab machine in order to access the data. To do this, run the cell below (you may need to click the "Restart Runtime" button when it finishes). You will have to do this every time you return to work on the homework.

In [1]:
!pip install torchdata==0.5.1
!pip install torchtext==0.14.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 58.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchdata
    Found existing installation: torchdata 0.6.1
    Uninstalling torchdata-0.6.1:
      Successfully uninstalled torchdata-0.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 

## Preprocessing the Data

To make your models more robust, it is necessary to perform some basic preprocessing on the corpora. <i>You do not need to edit this code.</i>

* <b>Sentence splitting:</b>&nbsp;&nbsp;&nbsp;&nbsp;In this homework, we are interested in modeling individual sentences, rather than longer chunks of text such as paragraphs or documents. The WikiTest dataset provides paragraphs; thus, we provide a simple method to identify individual sentences by splitting paragraphs at punctuation tokens (".",  "!",  "?").

* <b>Sentence markers:</b>&nbsp;&nbsp;&nbsp;&nbsp;For both training and testing corpora, each sentence must be surrounded by a start-of-sentence (`<s>`) and end-of-sentence marker (`/s`). These markers will allow your models to generate sentences that have realistic beginnings and endings.

* <b>Unknown words:</b>&nbsp;&nbsp;&nbsp;&nbsp;In order to deal with unknown words in the test corpora, all words that do not appear in the vocabulary must be replaced with a special token for unknown words (`<UNK>`) before estimating your models. The WikiText dataset has already done this, and you can read about the method in the paper above. When unknown words are encountered in the test corpus, they should be treated as that special token instead.

We provide you with preprocessing code here, and you should not modify it.

After the preprocessing, you may assume that all words in the test set appear in the training set, as this code has already replaced the unseen tokens with `<UNK>`.

In [2]:
# Constants (feel free to use these in your code, but do not change them)
START = "<s>"   # Start-of-sentence token
END = "</s>"    # End-of-sentence-token
UNK = "<UNK>"   # Unknown word token

In [3]:
### DO NOT EDIT ###

import torchtext
import random
import sys

def preprocess(data, vocab=None):
    final_data = []
    lowercase = "abcdefghijklmnopqrstuvwxyz"
    for paragraph in data:
        paragraph = [x if x != '<unk>' else UNK for x in paragraph.split()]
        if vocab is not None:
            paragraph = [x if x in vocab else UNK for x in paragraph]
        if paragraph == [] or paragraph.count('=') >= 2: continue
        sen = []
        prev_punct, prev_quot = False, False
        for word in paragraph:
            if prev_quot:
                if word[0] not in lowercase:
                    final_data.append(sen)
                    sen = []
                    prev_punct, prev_quot = False, False
            if prev_punct:
                if word == '"':
                    prev_punct, prev_quot = False, True
                else:
                    if word[0] not in lowercase:
                        final_data.append(sen)
                        sen = []
                        prev_punct, prev_quot = False, False
            if word in {'.', '?', '!'}: prev_punct = True
            sen += [word]
        if sen[-1] not in {'.', '?', '!', '"'}: continue # Prevent a lot of short sentences
        final_data.append(sen)
    vocab_was_none = vocab is None
    if vocab is None:
        vocab = set()
    for i in range(len(final_data)):
        final_data[i] = [START] + final_data[i] + [END]
        if vocab_was_none:
            for word in final_data[i]:
                vocab.add(word)
    return final_data, vocab

def getDataset():
    dataset = torchtext.datasets.WikiText2(root='.data', split=('train', 'valid'))
    train_dataset, vocab = preprocess(dataset[0])
    test_dataset, _ = preprocess(dataset[1], vocab)

    return train_dataset, test_dataset

if __name__ == '__main__':
    train_dataset, test_dataset = getDataset()

Run the next cell to see 10 random sentences of the training data.

In [4]:
if __name__ == '__main__':
    for x in random.sample(train_dataset, 10):
        print (x)

['<s>', 'Markgraf', 'was', 'ordered', 'under', 'the', 'provisional', 'name', 'Ersatz', '<UNK>', 'and', 'built', 'at', 'the', 'AG', '<UNK>', 'shipyard', 'in', '<UNK>', 'under', 'construction', 'number', '186', '.', '</s>']
['<s>', 'However', ',', 'as', 'the', 'group', "'s", 'Steeltown', 'contract', 'had', 'not', 'yet', 'expired', ',', 'the', 'new', 'contract', 'could', 'not', 'be', 'fully', 'executed', 'until', 'March', '11', ',', '1969', '.', '</s>']
['<s>', 'A', 'small', 'circulation', 'remained', 'in', 'the', 'cloud', 'field', 'offshore', 'the', 'northwest', 'coast', 'of', 'Baja', 'California', 'for', 'a', 'few', 'more', 'days', '.', '</s>']
['<s>', 'Wetherall', 'was', 'appointed', 'player', '@-@', 'manager', 'on', 'a', 'temporary', 'basis', 'and', 'then', 'for', 'the', 'rest', 'of', 'the', 'season', ',', 'but', 'City', 'were', 'relegated', 'following', 'a', '3', '–', '0', 'defeat', 'to', 'Chesterfield', '.', '</s>']
['<s>', 'Within', 'two', 'months', ',', 'most', 'juveniles', 'will'

## The LanguageModel Class

You will implement 4 types of language models: a <b>unigram</b> model, a <b>smoothed unigram</b> model, a <b>bigram</b> model, and a <b>smoothed bigram</b> model. Each of the models is worth 15 points and extends the following base class. <b>You do not need to implement anything in this class</b>; you will instead implement each of the following methods in the relevant subclass:

* <b>`__init__(self, trainCorpus)`</b>: Train the language model on `trainCorpus`. This will involve calculating relative frequency estimates according to the type of model you're implementing.

* <b>`generateSentence(self)`</b>: <b>[5 points]</b> Return a sentence that is generated by the language model. It should be a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>, where each <TT>w<sup>(i)</sup></TT> is a word in your vocabulary (including <TT>&lt;UNK&gt;</TT> but exlcuding <TT>&lt;s&gt;</TT> and <TT>&lt;&sol;s&gt;</TT>). You may assume that <TT>&lt;s&gt;</TT> starts each sentence (with probability $1$). The following words <TT>w<sup>(1)</sup></TT>, ... , <TT>w<sup>(n)</sup></TT>, <TT>&lt;&sol;s&gt;</TT> are generated according to your language model's distribution. Note that the number of words <TT>n</TT> is not fixed; instead, you should stop the sentence as soon as you generate the stop token <TT>&lt;&sol;s&gt;</TT>.

* <b>`getSentenceLogProbability(self, sentence)`</b>: <b>[5 points]</b> Return the <em> logarithm of the probability</em> of <TT>sentence</TT>, which is again a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>. You should use the natural logarithm $-$ that is, the base-<em>e</em> logarithm. See the note below about performing your calculations in log space.

* <b>`getCorpusPerplexity(self, testCorpus)`</b>: <b>[5 points]</b> You need to compute the perplexity (normalized inverse log probability) of `testCorpus` according to your model. For a corpus $W$ with $N$ words and a bigram model, Jurafsky and Martin tells you to compute perplexity as follows:

$$Perplexity(W) = \Big [ \prod_{i=1}^N \frac{1}{P(w^{(i)}|w^{(i-1)})} \Big ]^{1/N}$$

<b>Implementation Hint:</b> In order to avoid underflow, you will likely need to do all of your calculations in log-space. That is, instead of multiplying probabilities, you should add the logarithms of the probabilities and exponentiate the result:

$$\prod_{i=1}^N P(w^{(i)}|w^{(i-1)}) = \exp\Big (\sum_{i=1}^N \log P(w^{(i)}|w^{(i-1)}) \Big ) $$

Using this property should help you in your implementation of `generateSentence(self)` and `getCorpusPerplexity(self, testCorpus)`.

Feel free to implement helper methods as you wish (either in the base class or in the subclases). <b>But be sure not to change the function signatures of the provided methods</b> (i.e. the function and argument names), or else the autograder will fail.

In [5]:
import math
import random
from collections import defaultdict

class LanguageModel(object):
    def __init__(self, trainCorpus):
        '''
        Initialize and train the model (i.e. estimate the model's underlying probability
        distribution from the training corpus.)
        '''

        ### DO NOT EDIT ###
        return

    def generateSentence(self):
        '''
        Generate a sentence by drawing words according to the model's probability distribution.
        Note: Think about how to set the length of the sentence in a principled way.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement generateSentence in each subclass.")

    def getSentenceLogProbability(self, sentence):
        '''
        Calculate the log probability of the sentence provided.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement getSentenceProbability in each subclass.")

    def getCorpusPerplexity(self, testCorpus):
        '''
        Calculate the perplexity of the corpus provided.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement getCorpusPerplexity in each subclass.")

    def printSentences(self, n):
        '''
        Prints n sentences generated by your model.
        '''

        ### DO NOT EDIT ###
        for i in range(n):
            sent = self.generateSentence()
            prob = self.getSentenceLogProbability(sent)
            print('Log Probability:', prob , '\tSentence:',sent)

## <font color='red'>TODO:</font> Unigram Model [15 points]

Here, you will implement each of the 4 functions described above for an <b>unsmoothed unigram</b> model. The probability distribution of a word is given by $\hat P(w)$.

<font color='green'><b>Hints:</b></font>
* <font color='green'>You should use a <b>dictionary</b> to map tokens to their unigram counts.</font>
* <font color='green'>Since you never want to generate the start-of-sentence token `<s>`, you should <b>not</b> include it in your counts.</font>
* <font color='green'>In general, avoid checking for membership in a list (i.e. avoid `x in lst`). Instead, use sets or dictionaries for this purpose $-$ membership checks are much faster on these data structures.</font>
* <font color='green'>Do <b>not</b> modify the training or test corpora by using `.append(...)` or `.pop(...)` on them. This will cause unexpected behavior in the autograder tests, which do not expect you to be changing the data.

In [6]:
class UnigramModel(LanguageModel):
    def __init__(self, trainCorpus):
        self.unigram_counts = {}
        self.total_tokens = 0
        # Training on the corpus
        for sentence in trainCorpus:
          for token in sentence:
            if token != '<s>':
              self.unigram_counts[token] = self.unigram_counts.get(token, 0) + 1
              self.total_tokens += 1
    def generateSentence(self):
        generated_sentence = ["<s>"]
        token = None
        while token != "</s>":
          token = self.generateToken()
          generated_sentence.append(token)
        return generated_sentence


    def generateToken(self):
        sampled_word = random.choice(list(self.unigram_counts.keys()))
        while sampled_word == '<s>':
          sampled_word = random.choice(list(self.unigram_counts.keys()))
        return sampled_word

    def getSentenceLogProbability(self, sentence):
        log_prob = 0
        for token in sentence:
          if token not in self.unigram_counts and token != '<s>':
            # Handling words not in corpus. Probability will be zero.
            return float('-inf')
          elif token != '<s>':
            prob = self.unigram_counts[token] / self.total_tokens
            log_prob += math.log(prob)
        return log_prob

    def getCorpusPerplexity(self, testCorpus):
        total_log_prob = 0
        total_tokens = 0
        for sentence in testCorpus:
          total_log_prob += self.getSentenceLogProbability(sentence)
          total_tokens += len(sentence) - 1
          # Subtracting for <s> token
        return math.exp(-total_log_prob/total_tokens)

We provide you with a testing function that uses very simple training & test corpora (you could compute probability/perplexity by hand if you wanted to). This is just a <b>sanity check</b> $-$ passing this test does not guarantee you a perfect score in the autograder; this is simply to help you debug your model.

In [7]:
def sanityCheck(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}

    #	Read in the test corpus
    train_corpus = ["By the Late Classic , a network of few <unk> ( few <unk> ) linked various parts of the city , running for several kilometres through its urban core .",
    "Few people realize how difficult it was to create Sonic 's graphics engine , which allowed for the incredible rate of speed the game 's known for ."]
    test_corpus = ["Classic few parts of the game allowed for few <unk> <unk> incredible city .",
                   "Few <unk> realize the difficult network , which linked the game to Sonic ."]
    train_corpus, _ = preprocess(train_corpus)
    test_corpus, _ = preprocess(test_corpus)
    sentence = preprocess(["Sonic was difficult ."])[0][0]

    # These are the correct answers (don't change them!)
    if model_type == "unigram":
       senprobs = [-19.08542845, -114.5001481799, -108.7963657053, -53.6727664115, -55.4645258807]
       trainPerp, testPerp = 41.3308239726, 38.0122981569
       model = UnigramModel(train_corpus)
    elif model_type == "smoothed-unigram":
       senprobs = [-19.0405293515, -115.3479413049, -108.9114348746, -54.8190029616, -55.8122547346]
       trainPerp, testPerp = 41.9994393615, 39.9531928383
       model = SmoothedUnigramModel(train_corpus)
    elif model_type == "bigram":
       senprobs = [-float('inf'), -10.3450917073, -9.2464794186, -float('inf'), -float('inf')]
       trainPerp, testPerp = 1.3861445461, float('inf')
       model = BigramModel(train_corpus)
    elif model_type == "smoothed-bigram":
       senprobs = [-16.355820202, -76.0026113319, -74.2346475108, -47.2885760372, -51.2730261907]
       trainPerp, testPerp = 12.2307627397, 26.7193157699
       model = SmoothedBigramModelAD(train_corpus)
    else: assert False, 'Invalid model_type'

    print("--- TEST: generateSentence() ---")
    modelSen = model.generateSentence()
    senTestPassed = isinstance(modelSen, list) and len(modelSen) > 1 and isinstance(modelSen[0], str)
    if senTestPassed:
        print ("Test generateSentence() passed!")
    else:
        print ("Test generateSentence() failed; did not return a list of strings...")

    print("\n--- TEST: getSentenceLogProbability(...) ---")
    sentences = [sentence, *train_corpus, *test_corpus]
    failed = 0
    for i in range(len(sentences)):
        sen, correct_prob = sentences[i], senprobs[i]
        prob = round(model.getSentenceLogProbability(sen), 10)
        print("Correct log prob.:", correct_prob, '\tYour log prob.:', prob, '\t', 'PASSED' if prob == correct_prob else 'FAILED', '\t', sen)
        if prob != correct_prob: failed+=1

    if not failed:
        print ("Test getSentenceProbability(...) passed!")
    else:
        print("Test getSentenceProbability(...) failed on", failed, "sentence" if failed == 1 else 'sentences...')

    print("\n--- TEST: getCorpusPerplexity(...) ---")
    train_perp = round(model.getCorpusPerplexity(train_corpus), 10)
    test_perp = round(model.getCorpusPerplexity(test_corpus), 10)

    print("Correct train perp.:", trainPerp, '\tYour train perp.:', train_perp, '\t', 'PASSED' if trainPerp == train_perp else 'FAILED')
    print("Correct test perp.:", testPerp, '\tYour test perp.:', test_perp, '\t', 'PASSED' if testPerp == test_perp else 'FAILED')
    train_passed, test_passed = train_perp == trainPerp, test_perp == testPerp
    if train_passed and test_passed:
        print("Test getCorpusPerplexity(...) passed!")
    else:
        print("Test getCorpusPerplexity(...) failed on", "the training corpus and the testing corpus..." if not train_passed and not test_passed else "the testing corpus..." if not test_passed else "the training corpus...")

if __name__=='__main__':
    sanityCheck('unigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -19.08542845 	Your log prob.: -19.08542845 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -114.5001481799 	Your log prob.: -114.5001481799 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -108.7963657053 	Your log prob.: -108.7963657053 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -53.6727664115 	Your log prob.: -53.6727664115 	 PASSED 	 ['<s>', 'Classic', 'few', '

Next, we provide you with another <b>sanity check</b> that trains your model on the *entire* training set, and tests your functions on a small corpus (10 sentences) of *real* test data.

If your code is inefficient, you will likely see that this cell is taking too long. This cell is expected to run in fewer than <b>10 seconds</b>, so if it takes significantly longer than that, you should probably inspect your code for efficiency issues.

In [8]:
def sanityCheckFullDataset(model_type):
    model = UnigramModel(train_dataset)
    idxes = list(range(75,7500, 800))
    small_test_corpus = [test_dataset[idx] for idx in idxes]
    if model_type == 'unigram':
        senprobs = [-80.7782190984, -174.4769654449, -136.455148267, -225.5890741503, -719.0142129846, -236.350443633, -126.0056604204, -47.3424655612, -47.7775372096, -138.8159941929]
        testPerp = 881.0132848704
        model = UnigramModel(train_dataset)
    elif model_type == 'smoothed-unigram':
        senprobs = [-80.8423009715, -174.5131424172, -136.3181234818, -225.357454098, -719.1543898871, -236.6682968913, -126.1965419509, -47.4369338195, -47.7692144935, -138.542462715]
        testPerp = 881.6105352831
        model = SmoothedUnigramModel(train_dataset)
    elif model_type == 'bigram':
        senprobs = [-float('inf'), -float('inf'), -float('inf'), -float('inf'), -float('inf'), -float('inf'), -float('inf'), -32.1502020637, -float('inf'), -float('inf')]
        testPerp = float ('inf')
        model = BigramModel(train_dataset)
    elif model_type == 'smoothed-bigram':
        senprobs = [-61.3754065648, -141.9754903887, -107.0849366076, -168.4944718788, -619.9409055374, -195.8159911677, -86.3762008156, -32.4764801981, -48.124714509, -124.687107856]
        testPerp = 261.4247123506
        model = SmoothedBigramModelAD(train_dataset)
    else: assert False, 'Invalid model_type'
    print("\n--- TEST: getSentenceLogProbability(...) ---")
    failed = 0
    for i in range(len(small_test_corpus)):
        sen, correct_prob = small_test_corpus[i], senprobs[i]
        prob = round(model.getSentenceLogProbability(sen), 10)
        print("Correct log prob.:", correct_prob, '\tYour log prob.:', prob, '\t', 'PASSED' if prob == correct_prob else 'FAILED', '\t', sen)
        if prob != correct_prob: failed+=1

    if not failed:
        print ("Test getSentenceProbability(...) passed!")
    else:
        print("Test getSentenceProbability(...) failed on", failed, "sentence" if failed == 1 else 'sentences...')

    print("\n--- TEST: getCorpusPerplexity(...) ---")
    test_perp = round(model.getCorpusPerplexity(small_test_corpus), 10)

    print("Correct test perp.:", testPerp, '\tYour test perp.:', test_perp, '\t', 'PASSED' if testPerp == test_perp else 'FAILED')
    test_passed = test_perp == testPerp
    if test_passed:
        print("Test getCorpusPerplexity(...) passed!")
    else:
        print("Test getCorpusPerplexity(...) failed on the testing corpus...")

if __name__=='__main__':
    sanityCheckFullDataset('unigram')


--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -80.7782190984 	Your log prob.: -80.7782190984 	 PASSED 	 ['<s>', 'He', 'was', '<UNK>', 'at', '<UNK>', 'College', ',', 'Hobart', ',', 'and', '<UNK>', 'in', '1932', '.', '</s>']
Correct log prob.: -174.4769654449 	Your log prob.: -174.4769654449 	 PASSED 	 ['<s>', 'Despite', 'being', 'a', 'rare', 'Grade', '9', 'player', 'on', 'the', 'senior', 'team', ',', 'he', 'was', 'one', 'of', 'the', 'Knights', "'", 'two', 'leading', 'rushers', 'that', 'year', '.', '</s>']
Correct log prob.: -136.455148267 	Your log prob.: -136.455148267 	 PASSED 	 ['<s>', 'Burke', "'s", 'total', 'was', 'a', 'school', 'record', 'for', 'the', 'Big', 'Ten', 'Conference', 'Men', "'s", 'Basketball', 'Tournament', '.', '</s>']
Correct log prob.: -225.5890741503 	Your log prob.: -225.5890741503 	 PASSED 	 ['<s>', 'The', 'route', 'turns', 'to', 'the', 'northeast', ',', 'passing', 'near', 'the', '<UNK>', 'Leaf', 'Lakes', 'residential', 'development', ',', 'bef

Finally, you can train your model on the full WikiText corpus, and evaluate it on the held-out test set.

In [9]:
def runModel(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}
    # Read the corpora
    if model_type == 'unigram':
        model = UnigramModel(train_dataset)
    elif model_type == 'bigram':
        model = BigramModel(train_dataset)
    elif model_type == 'smoothed-unigram':
        model = SmoothedUnigramModel(train_dataset)
    else:
        model = SmoothedBigramModelAD(train_dataset)

    print("--------- 5 sentences from your model ---------")
    model.printSentences(5)

    print ("\n--------- Corpus Perplexities ---------")
    print ("Training Set:", model.getCorpusPerplexity(train_dataset))
    print ("Testing Set:", model.getCorpusPerplexity(test_dataset))

if __name__=='__main__':
    runModel('unigram')

--------- 5 sentences from your model ---------
Log Probability: -496423.357119547 	Sentence: ['<s>', 'moulded', 'Seminoles', 'Stelae', '460', 'stadiums', 'doubtful', 'Torchwood', 'fluency', 'CRIA', 'Cajal', 'dent', 'stores', '223', 'facility', 'Capel', 'hospitalization', 'campaigners', 'Legionnaires', 'Djan', 'Mickey', 'Signet', 'Folk', 'HURDAT', 'began', 'reward', 'stingray', 'scaling', 'Fatality', 'Andalee', 'Convict', 'facilitate', 'herders', 'giants', 'Non', 'Jubba', 'desire', 'bodily', 'unfold', 'sun', 'plasma', 'Rebellion', 'Figg', 'twice', 'provisional', '5A', 'asserted', 'Xenon', 'Ashton', 'Awards', 'recruits', 'Wally', 'happen', 'Uist', 'Jeremiah', 'Backed', 'outrageously', 'Kuehn', 'plume', 'nucleophilic', 'turnovers', 'Widmore', 'Juárez', 'Kookaburra', 'Gong', 'directorship', 'Cuney', 'Bessin', 'Hen', 'Fencing', 'fission', 'Apu', 'parties', 'disgusted', 'Mana', 'Rosebery', 'Shiina', 'reinforcements', 'accounting', 'clinic', 'promotion', 'judgment', 'mal', 'Breaking', 'extra

## <font color='red'>TODO:</font> Smoothed Unigram Model [15 points]

Here, you will implement each of the 4 functions described above for a <b>unigram</b> model with <b>Laplace (add-one) smoothing</b>. The probability distribution of a word is given by $P_L(w)$. This type of smoothing takes away some of the probability mass for observed events and assigns it to unseen events.

In order to smooth your model, you will need the number of words in the corpus, $N$, and the number of word types, $S$. The distinction between these is meaningful: $N$ indicates the number of word instances, where $S$ refers to the size of our vocabulary. For example, the sentence <em>the cat saw the dog</em> has four word types (<em>the</em>, <em>cat</em>, <em>saw</em>, <em>dog</em>), but five word tokens (<em>the</em>, <em>cat</em>, <em>saw</em>, <em>the</em>, <em>dog</em>). The token <em>the</em> appears twice in the sentence, but they share the same type <em>the</em>.

If $c(w)$ is the frequency of $w$ in the training data, you can compute $P_L(w)$ as follows:

$$P_L(w)=\frac{c(w)+1}{N+S}$$

<font color='green'><b>Hints:</b></font>
* <font color='green'>You may find it convenient to make your `SmoothedUnigramModel` inherit your `UnigramModel`, and then override the function(s) that need to be changed.</font>

In [10]:
class SmoothedUnigramModel(LanguageModel):
    def __init__(self, trainCorpus):
        self.unigram_counts = {}
        self.total_tokens = 0
        self.vocab_size = 0  # This will store the number of unique word types

        # Training on the corpus
        for sentence in trainCorpus:
            for token in sentence:
                if token != '<s>':
                    if token not in self.unigram_counts:
                        self.vocab_size += 1
                    self.unigram_counts[token] = self.unigram_counts.get(token, 0) + 1
                    self.total_tokens += 1

    def generateSentence(self):
        generated_sentence = ['<s>']
        token = None
        while token != '</s>':
            token = self.generateToken()
            generated_sentence.append(token)
        return generated_sentence

    def generateToken(self):
        sampled_word = random.choice(list(self.unigram_counts.keys()))
        while sampled_word == '<s>':
            sampled_word = random.choice(list(self.unigram_counts.keys()))
        return sampled_word

    def getSentenceLogProbability(self, sentence):
        log_prob = 0
        for token in sentence:
            if token != '<s>':
                prob = (self.unigram_counts.get(token, 0) + 1) / (self.total_tokens + self.vocab_size)
                log_prob += math.log(prob)
        return log_prob

    def getCorpusPerplexity(self, testCorpus):
        total_log_prob = 0
        total_tokens = 0
        for sentence in testCorpus:
            total_log_prob += self.getSentenceLogProbability(sentence)
            total_tokens += len(sentence) - 1  # Subtracting for <s> token
        return math.exp(-total_log_prob/total_tokens)

In [11]:
if __name__=='__main__':
    sanityCheck('smoothed-unigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -19.0405293515 	Your log prob.: -19.0405293515 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -115.3479413049 	Your log prob.: -115.3479413049 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -108.9114348746 	Your log prob.: -108.9114348746 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -54.8190029616 	Your log prob.: -54.8190029616 	 PASSED 	 ['<s>', 'Classic', 'few

Since the next sanity check trains your model on the *entire* training set, you will likely see that it is taking too long if you have inefficiences in your code. This cell is expected to run in fewer than <b>10 seconds</b>, so if it takes significantly longer than that, you should probably inspect your code for efficiency issues.

In [12]:
if __name__=='__main__':
    sanityCheckFullDataset('smoothed-unigram')


--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -80.8423009715 	Your log prob.: -80.8423009715 	 PASSED 	 ['<s>', 'He', 'was', '<UNK>', 'at', '<UNK>', 'College', ',', 'Hobart', ',', 'and', '<UNK>', 'in', '1932', '.', '</s>']
Correct log prob.: -174.5131424172 	Your log prob.: -174.5131424172 	 PASSED 	 ['<s>', 'Despite', 'being', 'a', 'rare', 'Grade', '9', 'player', 'on', 'the', 'senior', 'team', ',', 'he', 'was', 'one', 'of', 'the', 'Knights', "'", 'two', 'leading', 'rushers', 'that', 'year', '.', '</s>']
Correct log prob.: -136.3181234818 	Your log prob.: -136.3181234818 	 PASSED 	 ['<s>', 'Burke', "'s", 'total', 'was', 'a', 'school', 'record', 'for', 'the', 'Big', 'Ten', 'Conference', 'Men', "'s", 'Basketball', 'Tournament', '.', '</s>']
Correct log prob.: -225.357454098 	Your log prob.: -225.357454098 	 PASSED 	 ['<s>', 'The', 'route', 'turns', 'to', 'the', 'northeast', ',', 'passing', 'near', 'the', '<UNK>', 'Leaf', 'Lakes', 'residential', 'development', ',', 'bef

In [13]:
if __name__=='__main__':
    runModel('smoothed-unigram')

--------- 5 sentences from your model ---------
Log Probability: -12330.09580950968 	Sentence: ['<s>', 'Merrifield', 'Veerashaiva', 'Choir', 'biologist', 'Brandt', 'dumped', 'rewritten', 'compile', 'McCall', 'dredged', 'Region', 'impacting', 'pulpit', 'forward', 'aircraft', 'conversations', 'Olav', 'varieties', 'Natalie', 'Free', 'Grove', 'HaMifratz', 'preyed', 'stirring', 'Kepler', 'bivouac', 'Demolition', 'criticizing', 'Den', 'Mantell', 'Myth', 'Demolition', 'goals', 'Ke', 'Bedell', 'Kanye', 'murals', '″', 'Torchwood', 'tis', 'residing', 'photos', 'Foxes', 'Hibari.Ch.', 'portable', 'Short', 'UTC', 'Notes', 'wardrobe', 'lamps', 'Madsen', 'lirpa', 'dream', 'Bengali', 'fills', '1161', 'explanations', '2019', 'Highlands', 'Lacey', 'Judgment', 'Aegean', 'planted', 'Deva', 'pave', 'organizer', 'Wine', 'ibotenic', 'fortune', 'Curtain', 'scholastic', 'Pro40', 'competent', 'clause', 'lectures', 'ambassadorial', 'Treason', 'Bulgarian', 'tertiary', 'staff', 'sortie', 'Sustainability', 'accusat